In [32]:
#importing processing packages
import pandas as pd 
import numpy as np 
import missingno as miss
import os
import warnings
import re
from nltk.corpus import words
import nltk

# Download the NLTK words corpus (only need to do this once)
nltk.download('words')
warnings.filterwarnings("ignore")

#importing viz packages
import plotly.express as px
import plotly 
import matplotlib.pyplot as plt

#importing machine learning packages
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

[nltk_data] Downloading package words to
[nltk_data]     /Users/ellisobrien/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [34]:
# Define the directory path where the JSON files are located
directory = '/Users/ellisobrien/Desktop/Kaggle/Educational Data/pii-detection-removal-from-educational-data/'

# Define the filenames for training and testing data
train_file = 'train.json'
test_file = 'test.json'

# Construct the full file paths
train_filepath = directory + train_file
test_filepath = directory + test_file

# Read the training data from JSON
df_train = pd.read_json(train_filepath)

# Read the testing data from JSON
df_test = pd.read_json(test_filepath)

In [36]:
# Load the English words corpus from NLTK
english_words = set(words.words())

# Function to preprocess text data and filter out non-words
def preprocess_and_filter(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters, punctuation, and extra whitespace
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenize the text into words
    tokens = text.split()
    # Filter out non-words
    filtered_tokens = [token for token in tokens if token in english_words]
    return filtered_tokens

# Apply the preprocess_and_filter function to the 'full_text' column
df_train['filtered_tokens'] = df_train['full_text'].apply(preprocess_and_filter)

# Display the DataFrame with filtered tokens
print(df_train[['document', 'full_text', 'filtered_tokens']])

      document                                          full_text  \
0            7  Design Thinking for innovation reflexion-Avril...   
1           10  Diego Estrada\n\nDesign Thinking Assignment\n\...   
2           16  Reporting process\n\nby Gilberto Gamboa\n\nCha...   
3           20  Design Thinking for Innovation\n\nSindy Samaca...   
4           56  Assignment:  Visualization Reflection  Submitt...   
...        ...                                                ...   
6802     22678  EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE    My w...   
6803     22679  Why Mind Mapping?\n\nMind maps are graphical r...   
6804     22681  Challenge\n\nSo, a few months back, I had chos...   
6805     22684  Brainstorming\n\nChallenge & Selection\n\nBrai...   
6806     22687  Mind Mapping\n\nChallenge\n\nMy consulting tea...   

                                        filtered_tokens  
0     [design, thinking, for, innovation, challenge,...  
1     [design, thinking, assignment, visualization, ...

In [39]:
# Filter the DataFrame to show only rows where potential_usernames is not blank
non_empty_usernames_df = df_train[df_train['filtered_tokens'].apply(lambda x: len(x) > 0)]

# Display the filtered DataFrame
print(non_empty_usernames_df[['full_text', 'filtered_tokens']])

                                              full_text  \
0     Design Thinking for innovation reflexion-Avril...   
1     Diego Estrada\n\nDesign Thinking Assignment\n\...   
2     Reporting process\n\nby Gilberto Gamboa\n\nCha...   
3     Design Thinking for Innovation\n\nSindy Samaca...   
4     Assignment:  Visualization Reflection  Submitt...   
...                                                 ...   
6802  EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE    My w...   
6803  Why Mind Mapping?\n\nMind maps are graphical r...   
6804  Challenge\n\nSo, a few months back, I had chos...   
6805  Brainstorming\n\nChallenge & Selection\n\nBrai...   
6806  Mind Mapping\n\nChallenge\n\nMy consulting tea...   

                                        filtered_tokens  
0     [design, thinking, for, innovation, challenge,...  
1     [design, thinking, assignment, visualization, ...  
2     [process, by, challenge, i, received, a, promo...  
3     [design, thinking, for, innovation, university...  
4

In [ ]:
# # Function to identify potential usernames in a given text
# def find_usernames(text):
#     # Define regular expression pattern for identifying potential usernames
#     username_pattern = r'\b@\w{3,20}\b'
    
#     # Find all occurrences of potential usernames in the text
#     potential_usernames = re.findall(username_pattern, text)
    
#     return potential_usernames

# # Apply the find_usernames function to each row in the DataFrame
# df_train['potential_usernames'] = df_train['full_text'].apply(find_usernames)

# # Display the DataFrame with potential usernames identified
# print(df_train[['full_text', 'potential_usernames']])


# # Function to preprocess text data
# def preprocess_text(text):
#     # Convert text to lowercase
#     text = text.lower()
#     # Remove special characters, punctuation, and extra whitespace
#     text = re.sub(r'[^\w\s]', '', text)
#     text = re.sub(r'\s+', ' ', text).strip()
#     # Tokenize the text into words
#     tokens = text.split()
#     return tokens

# # Apply the preprocess_text function to the 'full_text' column
# df_train['tokens'] = df_train['full_text'].apply(preprocess_text)

# # Display the DataFrame with tokens
# print(df_train[['document', 'full_text', 'tokens']])